<a href="https://colab.research.google.com/github/mgk0422/python-basic/blob/master/final_project09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import time
import itertools
import scipy
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
양파_반입량=pd.read_excel('/content/양파 반입량 통합2013-2020.xlsx')
양파_경락가=pd.read_excel('/content/양파(특) 경락가 통합2013-2020.xlsx')
양파_해남_기상=pd.read_excel('/content/양파_해남20130101_20201031.xlsx')
주유소_판매가격=pd.read_excel('/content/주유소_평균판매가격.xlsx')
양파_반입량.head()

In [ ]:
양파_경락가.info()

In [ ]:
양파_경락_반입=pd.merge(양파_반입량,양파_경락가,on='일자')
양파_경락_반입.head()

In [ ]:
양파_경락_반입['일자'] = 양파_경락_반입['일자'].astype(str)
양파_경락_반입['일자'] = pd.to_datetime(양파_경락_반입['일자'])
양파_경락_반입.info()

In [ ]:
#양파_해남_기상.head()

In [ ]:
양파_해남_기상.rename(columns = {'일시': '일자'}, inplace = True)

In [ ]:
양파_dt=pd.merge(양파_해남_기상,양파_경락_반입,on='일자')

In [ ]:
#양파_dt.head()

In [ ]:
양파_dt=pd.merge(양파_dt,주유소_판매가격,on='일자')

In [ ]:
양파_dt.head()

In [ ]:
양파_dt[['일교차']]=양파_dt['최고기온(°C)']-양파_dt['최저기온(°C)']

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','반입량','유가 전국평균가격','일교차','평균가격']]

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [ ]:
X, y = 양파_dt.iloc[:,:-1],양파_dt.iloc[:,-1]

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg.fit(X_train,y_train)

In [ ]:
preds = xg_reg.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,'max_depth': 5, 'alpha': 10}
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

In [ ]:
cv_results.head()

In [ ]:
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [40, 10]
plt.show()

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','평균 지면온도(°C)','반입량','평균가격','유가 전국평균가격']]

In [ ]:
양파_dt.info()

In [ ]:
# 설명변수(X),타겟변수(Y) 분리 및 학습 데이터와 평가 데이터 분할
feature_columns=list(양파_dt.columns.difference(['평균가격']))
X=양파_dt[feature_columns]
y=양파_dt.평균가격
train_x,test_x,train_y,test_y=train_test_split(X,y,train_size=0.7,test_size=0.3)
print(train_x.shape,test_x.shape,train_y.shape,test_y.shape)

In [ ]:
#Train the MLR/회귀모델적합
full_model=sm.OLS(train_y,train_x)
fitted_full_model=full_model.fit()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_dt.values,i) for i in range(양파_dt.shape[1])]
vif['features']=양파_dt.columns
vif

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','최고기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','반입량','평균가격','유가 전국평균가격']]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_dt.values,i) for i in range(양파_dt.shape[1])]
vif['features']=양파_dt.columns
vif

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','일강수량(mm)','평균 풍속(m/s)','평균 상대습도(%)','합계 일조시간(hr)','반입량','평균가격','유가 전국평균가격']]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_dt.values,i) for i in range(양파_dt.shape[1])]
vif['features']=양파_dt.columns
vif

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','일강수량(mm)','평균 풍속(m/s)','합계 일조시간(hr)','반입량','유가 전국평균가격']]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=pd.DataFrame()
vif['VIF Factor']=[variance_inflation_factor(양파_dt.values,i) for i in range(양파_dt.shape[1])]
vif['features']=양파_dt.columns
vif

In [ ]:
# 변수선택법 
def processSubset(X, y, feature_set):
    model = sm.OLS(y, X[list(feature_set)]) # modeling
    regr = model.fit() # 모델 학습
    AIC = regr.aic # 모델의 AIC
    return {'model' : regr, 'AIC' : AIC}

print(processSubset(X=train_x, y = train_y, feature_set = feature_columns[0:5]))

In [ ]:
# 전진선택법(step=1)

def forward(X, y, predictors):
    # 데이터 변수들이 미리 정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in X.columns.difference(['평균가격']) if p not in predictors]
    tic = time.time()
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X=X, y=y, feature_set=predcitors+[p]+['평균가격']))
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)
    
    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic))
    print('Selected predictors:', best_model['model'].model.exog_names, ' AIC:',best_model[0])
    return best_model

In [ ]:
# 전진선택법 모델
def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=['AID', 'model'])
    tic = time.time()
    # 미리 정의된 데이터 변수
    predictors = []
    # 변수 1~10개 : 0 ~ 9  -> 1 ~ 10
    for i in range(1, len(X.columns.difference(['평균가격'])) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != '평균가격']
    toc = time.time()
    print("Total elapsed time : ", (toc - tic), "seconds.")
    
    return (Fmodels['model'][len(Fmodels['model'])])

In [ ]:
#Forward_best_model = forward_model(X=train_x, y=train_y)

In [ ]:
양파_dt=양파_dt[['최저기온(°C)','일강수량(mm)','평균 풍속(m/s)','합계 일조시간(hr)','반입량','유가 전국평균가격','평균가격']]
양파_dt.head()

In [ ]:
from sklearn import linear_model

X_data=양파_dt[['최저기온(°C)','일강수량(mm)','평균 풍속(m/s)','합계 일조시간(hr)','반입량','유가 전국평균가격']]
Y_data=양파_dt[['평균가격']]

In [ ]:
#다중선형회귀모델 설계
linear_regression_model=linear_model.LinearRegression()
linear_regression_model.fit(X_data,Y_data)
linear_regression_model_prediction=linear_regression_model.predict(X=pd.DataFrame(X_data))

In [ ]:
#Train the MLR / 회귀모델적합
fullModel = sm.OLS(train_y,train_x)
fittedFullModel = fullModel.fit()
 
#R-Squre 가 높고 , 대부분의 변수들이 유의함.
print(fittedFullModel.summary())
 
#VIF를 통한 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
 
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(양파_dt.values,i)
                     for i in range(양파_dt.shape[1])]
vif["features"]=양파_dt.columns
print(vif)


In [ ]:
#학습데이터의 잔차 확인
 
res = fittedFullModel.resid
import matplotlib.pyplot as plt
#Q-Q plot # 정규분포확인
fig = sm.qqplot(res, fit=True, line='45')
plt.show()
# residual pattern 확인
 
predY = fittedFullModel.predict(train_x)
 
fig = plt.scatter(predY,res,s=4)
plt.xlim(4000,30000)
plt.xlim(4000,30000)
plt.xlabel('Fitted values')
plt.ylabel('Residual')
plt.show()
 
#검증 데이터에 대한 예측
 
predY2 = fittedFullModel.predict(test_x)
 
plt.plot(np.array(test_y-predY2),label="predFull")
plt.legend()
plt.show()
 
#MSE 값 구하기
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_true=test_y,y_pred=predY2)
print(MSE)